# Read the Pushshift Telegram Dataset (also exists for Reddit)

In [2]:
# %pip install --user zstandard

In [3]:
import pandas as pd
import zstandard as zstd
import json

In [4]:
import os
os.getcwd()

'C:\\Users\\HP\\PhD\\PushshiftChannels'

In [5]:
# Create Zreader class because Zreader module will not load (https://github.com/pushshift/zreader)

class Zreader:

    def __init__(self, file, chunk_size=16384):
        '''Init method'''
        self.fh = open(file,'rb')
        self.chunk_size = chunk_size
        self.dctx = zstd.ZstdDecompressor()
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''


    def readlines(self):
        '''Generator method that creates an iterator for each line of JSON'''
        while True:
            chunk = self.reader.read(self.chunk_size).decode('utf-8', 'ignore')
            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            self.buffer = lines[-1]

Channels

In [9]:
# Channels


file = "D:\._PhD\Literature\Telegram\Data\The Pushshift Telegram Dataset\channels.ndjson.zst"
# Adjust chunk_size as necessary -- defaults to 16,384 if not specified
reader = Zreader(file)

# Read each line from the reader
channels = {}
i=0
for line in reader.readlines():
    obj = json.loads(line)
    channels[i] = obj
#     print(obj)
    i+=1

In [57]:
len(channels)

27946

Messages

In [53]:
# # Messages

# file = "D:\._PhD\Literature\Telegram\Data\The Pushshift Telegram Dataset\messages.ndjson.zst"
# # Adjust chunk_size as necessary -- defaults to 16,384 if not specified
# reader = Zreader(file, chunk_size=3)

# # Read each line from the reader
# messages = {}
# i=0
# for line in reader.readlines():
#     obj = json.loads(line)
#     messages[i] = obj
#     # print(obj)
#     i+=1

In [54]:
import ujson
import zstandard

# Define the features you're interested in
features_of_interest = ['to_id', 'fwd_from', 'id', 'message']

# Create an empty dictionary to store the filtered records
filtered_records = {}

# Open the compressed file for reading
with open('D:\._PhD\Literature\Telegram\Data\The Pushshift Telegram Dataset\messages.ndjson.zst', 'rb') as compressed_file:
    # Create a zstd decompressor
    decompressor = zstandard.ZstdDecompressor()

    # Open a decompression stream
    with decompressor.stream_reader(compressed_file) as reader:
        buffer = b""  # Initialize an empty buffer to accumulate data
        
        while True:
            # Read a chunk of data
            chunk = reader.read(4096)  # Adjust buffer size as needed
            
            # If no more data is available, break the loop
            if not chunk:
                break
            
            buffer += chunk  # Append the chunk to the buffer
            
            # Split the buffer into lines
            while b'\n' in buffer:
                # Extract the next line from the buffer
                line, buffer = buffer.split(b'\n', 1)
                
                # Decode the line from bytes to string
                line_str = line.decode('utf-8')
                
                try:
                    # Parse the JSON record using ujson
                    record = ujson.loads(line_str)
                    
                    # Filter the record to include only the features of interest
                    filtered_record = {key: record[key] for key in features_of_interest if key in record}
                    
                    # Store the filtered record in the dictionary
                    # You can choose a unique identifier for each record as the key
                    # Here, we're using the index of the record as the key
                    filtered_records[len(filtered_records)] = filtered_record
                except ValueError:
                    print("Error decoding JSON:", line_str)  # Handle JSON decoding errors



KeyboardInterrupt



In [56]:
len(filtered_records)

5036723

In [48]:
filtered_records[0]

[{'_': 'PeerChannel', 'channel_id': 1271719213},
 None,
 23831,
 'https://www.bitchute.com/video/VjCxvmWj6mvG/']